In [3]:
# Import our dependencies
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

# File to Load (Remember to Change These)
train_data = "cleaned_ASQ_train.csv"

In [4]:
df_train_clean = pd.read_csv(train_data)

In [5]:
df_train_clean[0:5]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,ASQ_Total_score,Autism_Diagnosis
0,1,0,1,1,1,1,0,1,1,1,7.819715,0
1,0,0,0,0,0,0,0,0,0,1,10.544296,0
2,1,1,1,1,1,1,0,0,1,1,13.167506,1
3,0,0,0,1,0,0,0,0,0,0,1.530098,0
4,0,0,0,0,1,0,0,0,1,1,7.949723,0


In [6]:
# Split our preprocessed data into our features and target arrays
y = df_train_clean.Autism_Diagnosis.values
X = df_train_clean.drop(columns="Autism_Diagnosis").values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ROS = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled= ROS.fit_resample(X_train_scaled, y_train)

In [9]:
# Count the distinct values of the resampled labels data
y_resampled[:5]

array([0, 1, 0, 1, 0])

In [10]:
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=1)

In [11]:
series = pd.Series(y_resampled)
series.value_counts()

0    464
1    464
dtype: int64

In [12]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.1 MB/s eta 0:00:00


In [13]:
# Create a method that creates a new Sequential model with hyperparameter options
input_features = X_train_scaled.shape[1]

def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value= 150,
        step=10), activation=activation, input_dim=input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=100,
            step=10),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

  # Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test))

# Top 3 model hyperparameters 
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
  print(param.values)

# Top 3 models
top_model = tuner.get_best_models(3)
for model in top_model:
  model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
  print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters()[0]
best_hyper.values

# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")



Trial 180 Complete [00h 00m 07s]
val_accuracy: 0.8836206793785095

Best val_accuracy So Far: 0.9612069129943848
Total elapsed time: 00h 11m 07s
{'activation': 'relu', 'first_units': 41, 'num_layers': 7, 'units_0': 21, 'units_1': 91, 'units_2': 51, 'units_3': 41, 'units_4': 51, 'units_5': 71, 'units_6': 91, 'units_7': 71, 'units_8': 21, 'tuner/epochs': 50, 'tuner/initial_epoch': 17, 'tuner/bracket': 3, 'tuner/round': 3, 'tuner/trial_id': '0047', 'units_9': 71}
{'activation': 'relu', 'first_units': 91, 'num_layers': 8, 'units_0': 11, 'units_1': 81, 'units_2': 51, 'units_3': 81, 'units_4': 61, 'units_5': 81, 'units_6': 51, 'units_7': 41, 'units_8': 61, 'units_9': 81, 'tuner/epochs': 50, 'tuner/initial_epoch': 17, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0069'}
{'activation': 'relu', 'first_units': 101, 'num_layers': 10, 'units_0': 41, 'units_1': 1, 'units_2': 81, 'units_3': 81, 'units_4': 61, 'units_5': 41, 'units_6': 31, 'units_7': 41, 'units_8': 41, 'units_9': 21, 'tuner

In [14]:
# Export our model to HDF5 file
best_model.save("Neural_Network_Optimized_Resampled.h5")